In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
from sklearn.metrics import mean_squared_error
np.random.seed(0)
import os

In [3]:
# IQR function outliers removal
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    new_arr = ys[(ys > upper_bound) | (ys < lower_bound)]
    return new_arr

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
features = [
    'galactic year',
    'galaxy',
    'existence expectancy index',
     'existence expectancy at birth',
     'Gross income per capita',
     'Income Index',
     'Expected years of education (galactic years)',
     'Mean years of education (galactic years)',
     'Intergalactic Development Index (IDI)',
     'Education Index',
     'Intergalactic Development Index (IDI), Rank',
     'Population using at least basic drinking-water services (%)',
     'Population using at least basic sanitation services (%)'
]

In [6]:
features_nan = [
 'existence expectancy index',
 'existence expectancy at birth',
 'Gross income per capita',
 'Income Index',
 'Expected years of education (galactic years)',
 'Mean years of education (galactic years)',
 'Intergalactic Development Index (IDI)',
 'Education Index',
 'Intergalactic Development Index (IDI), Rank',
 'Population using at least basic drinking-water services (%)',
 'Population using at least basic sanitation services (%)'
]

In [7]:
# Drop galactics not in test
gal_drop = train[
    (train.galaxy == 'Tucana Dwarf') |
    (train.galaxy == 'Andromeda XXII[57]') |
    (train.galaxy == 'Andromeda XVIII[60]') |
    (train.galaxy == 'Triangulum Galaxy (M33)') |
    (train.galaxy == 'Andromeda XXIV') |
    (train.galaxy == 'Andromeda XII') |
    (train.galaxy == 'NGC 5253') |
    (train.galaxy == 'Andromeda XIX[60]') |
    (train.galaxy == 'Hercules Dwarf')
]

train = train.drop(gal_drop.index)
target = 'y'

tr_features = [ col for col in train.columns if col  in features+[target]]
train = train[tr_features].reset_index(drop=True)
test = test[features]

train.head(2)

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),y
0,990025,Large Magellanic Cloud (LMC),0.628657,63.125200,27109.234310,0.646039,8.240543,NaN,NaN,NaN,NaN,NaN,NaN,0.052590
1,990025,Camelopardalis B,0.818082,81.004994,30166.793958,0.852246,10.671823,4.74247,0.833624,0.467873,152.522198,NaN,NaN,0.059868


In [9]:
train.shape, test.shape

((3664, 14), (890, 13))

In [10]:
train = train.fillna(train.groupby('galaxy')[features_nan].transform('mean'))
test = test.fillna(test.groupby('galaxy')[features_nan].transform('mean'))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3664 entries, 0 to 3663
Data columns (total 14 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   galactic year                                                3664 non-null   int64  
 1   galaxy                                                       3664 non-null   object 
 2   existence expectancy index                                   3664 non-null   float64
 3   existence expectancy at birth                                3664 non-null   float64
 4   Gross income per capita                                      3664 non-null   float64
 5   Income Index                                                 3664 non-null   float64
 6   Expected years of education (galactic years)                 3664 non-null   float64
 7   Mean years of education (galactic years)                     3664 non-null   f

In [11]:
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3664 entries, 0 to 3663
Data columns (total 14 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   galactic year                                                3664 non-null   int64  
 1   galaxy                                                       3664 non-null   object 
 2   existence expectancy index                                   3664 non-null   float64
 3   existence expectancy at birth                                3664 non-null   float64
 4   Gross income per capita                                      3664 non-null   float64
 5   Income Index                                                 3664 non-null   float64
 6   Expected years of education (galactic years)                 3664 non-null   float64
 7   Mean years of education (galactic years)                     3664 non-null   f

In [12]:
# New Features grouped by galaxy based on target
dd = pd.DataFrame()
groups = train.groupby('galaxy')
for name, group in groups:
    
    grouped = group.sort_values('galactic year')
        
    sample = group.y
    mean = group['y'].mean()
    median = group['y'].median()
    
    iqr_sample = outliers_iqr(sample)   
    new_sample = sample.drop(iqr_sample.index)
 
    log = np.log1p(new_sample)
    
    dd = dd.append({'galaxy': name,
                                       
                    'y_mean': mean,
                    'y_median': median,
                    'y_min': sample.min(),
                    'y_max': sample.max(),
                    'y_std': sample.std(),
                    'y_var': sample.var(),
                                                            
                    'y_mean_log': log.mean(),
                    'y_median_log': log.median(),
                    'y_min_log': log.std(),
                    'y_max_log': log.std(),
                    'y_std_log': log.std(),
                    'y_var_log': log.var(),
                                        
                    'y_quan_log_25': np.quantile(log,0.25),
                    'y_quan_log_75': np.quantile(log,0.75),
                    'y_quan_log_95': np.quantile(log,0.95),
                    'y_quan_log_99': np.quantile(log,0.99),
                    'IQR': (np.quantile(log,0.75) - np.quantile(log,0.25)),
                                 
                    
                   }, ignore_index=True)
dd.head()

,IQR,galaxy,y_max,y_max_log,y_mean,y_mean_log,y_median,y_median_log,y_min,y_min_log,y_quan_log_25,y_quan_log_75,y_quan_log_95,y_quan_log_99,y_std,y_std_log,y_var,y_var_log
0,0.005321,Andromeda Galaxy (M31),0.049214,0.003044,0.043539,0.044118,0.045632,0.044788,0.016824,0.003044,0.041349,0.046671,0.047528,0.047938,0.007346,0.003044,0.000054,0.000009
1,0.003206,Andromeda I,0.053204,0.001871,0.049126,0.049260,0.050877,0.049757,0.025844,0.001871,0.047525,0.050731,0.051396,0.051749,0.006115,0.001871,0.000037,0.000003
2,0.014508,Andromeda II,0.191913,0.007321,0.176689,0.165078,0.175048,0.162243,0.143884,0.007321,0.157539,0.172047,0.175347,0.175525,0.012812,0.007321,0.000164,0.000054
3,0.005476,Andromeda III,0.137202,0.003507,0.117387,0.109943,0.115198,0.108828,0.110758,0.003507,0.107710,0.113186,0.114775,0.116257,0.006237,0.003507,0.000039,0.000012
4,0.010048,Andromeda IX,0.285940,0.005670,0.166813,0.148237,0.159053,0.147307,0.151314,0.005670,0.143416,0.153464,0.155916,0.158846,0.030409,0.005670,0.000925,0.000032


In [13]:
df_train = pd.merge(train,dd,on='galaxy', how='left')
df_train.head()

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,y_min,y_min_log,y_quan_log_25,y_quan_log_75,y_quan_log_95,y_quan_log_99,y_std,y_std_log,y_var,y_var_log
0,990025,Large Magellanic Cloud (LMC),0.628657,63.125200,27109.234310,0.646039,8.240543,5.838158,0.743050,0.558598,...,0.031360,0.001892,0.046872,0.050060,0.050892,0.051181,0.006028,0.001892,0.000036,0.000004
1,990025,Camelopardalis B,0.818082,81.004994,30166.793958,0.852246,10.671823,4.742470,0.833624,0.467873,...,0.051950,0.001439,0.055701,0.057641,0.058204,0.058252,0.001981,0.001439,0.000004,0.000002
2,990025,Virgo I,0.659443,59.570534,8441.707353,0.499762,8.840316,5.583973,0.469110,0.363837,...,0.018371,0.005349,0.039455,0.046538,0.048208,0.049000,0.007463,0.005349,0.000056,0.000029
3,990025,UGC 8651 (DDO 181),0.555862,52.333293,21887.819939,0.600629,8.805304,8.673661,0.583649,0.492168,...,0.016685,0.004873,0.038850,0.046090,0.047387,0.048041,0.008457,0.004873,0.000072,0.000024
4,990025,KKh 060,0.824692,63.887135,28409.062695,0.671697,14.062458,9.978597,0.815264,0.796807,...,0.022488,0.003296,0.043832,0.050004,0.050854,0.051387,0.009378,0.003296,0.000088,0.000011


In [14]:
df_test = pd.merge(test,dd,on='galaxy', how='left')
df_test.head()

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,y_min,y_min_log,y_quan_log_25,y_quan_log_75,y_quan_log_95,y_quan_log_99,y_std,y_std_log,y_var,y_var_log
0,1007012,KK98 77,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,...,0.025967,0.003271,0.043740,0.049502,0.050798,0.051891,0.006761,0.003271,0.000046,0.000011
1,1007012,Reticulum III,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,...,0.019159,0.003145,0.041721,0.047234,0.048181,0.048729,0.008731,0.003145,0.000076,0.000010
2,1008016,Reticulum III,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,...,0.019159,0.003145,0.041721,0.047234,0.048181,0.048729,0.008731,0.003145,0.000076,0.000010
3,1007012,Segue 1,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,...,0.020578,0.003030,0.041601,0.046754,0.047651,0.048226,0.008151,0.003030,0.000066,0.000009
4,1013042,Virgo I,0.588274,55.428320,23959.704016,0.520579,10.392416,6.374637,0.530676,0.580418,...,0.018371,0.005349,0.039455,0.046538,0.048208,0.049000,0.007463,0.005349,0.000056,0.000029


In [15]:
combined = pd.concat([df_train,df_test], axis=0)

In [16]:
# Some more features
groups = combined.groupby(['galaxy'])
i = 0
index= []

grouped_data = pd.DataFrame()
for gr in groups.groups:

    #print(groups.get_group(gr).sort_values('galactic year'))
    grouped = groups.get_group(gr).sort_values('galactic year')
   
    grouped['ei_mean'] = np.nanmean(grouped['existence expectancy index'])
    grouped['gross_mean'] = np.nanmean(grouped['Gross income per capita'])
    grouped['dev_mean'] = np.nanmean(grouped['Intergalactic Development Index (IDI)'])
    grouped['in_idx'] = np.nanmean(grouped['Income Index'])
    
    grouped['nan_y_quan_log_25'] =  np.nanquantile(np.log1p(grouped.y),0.25)
    grouped['nan_y_quan_log_75']= np.nanquantile(np.log1p(grouped.y),0.75)
    grouped['nan_y_quan_log_95']= np.nanquantile(np.log1p(grouped.y),0.95)
    grouped['nan_y_quan_log_99']= np.nanquantile(np.log1p(grouped.y),0.99)
    
    grouped['nan_y_mean_log'] =  np.nanmean(np.log1p(grouped.y))
    grouped['nan_y_median_log']= np.nanmedian(np.log1p(grouped.y))
    grouped['nan_y_std_log']= np.nanstd(np.log1p(grouped.y))
    

    index.append(grouped.index)
    grouped_data = grouped_data.append(grouped)
    

In [17]:
dtest = grouped_data[grouped_data.y.isnull() == True]
dtrain = grouped_data[grouped_data.y.isnull() == False]

In [18]:
#dtrain.fillna(method='bfill', inplace=True)
#dtest.fillna(method='bfill', inplace=True)

In [19]:
dtrain.head()

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,...,gross_mean,dev_mean,in_idx,nan_y_quan_log_25,nan_y_quan_log_75,nan_y_quan_log_95,nan_y_quan_log_99,nan_y_mean_log,nan_y_median_log,nan_y_std_log
120,990025,Andromeda Galaxy (M31),0.759989,72.020628,18445.323465,0.647025,13.819287,10.302484,0.724764,0.770957,...,20480.025457,0.735599,0.688991,0.041153,0.046601,0.047496,0.047932,0.042594,0.044622,0.006908
229,991020,Andromeda Galaxy (M31),0.769566,68.279470,17844.399709,0.678283,15.518075,10.302484,0.724764,0.770957,...,20480.025457,0.735599,0.688991,0.041153,0.046601,0.047496,0.047932,0.042594,0.044622,0.006908
445,992016,Andromeda Galaxy (M31),0.791677,69.154885,17407.794954,0.658309,14.660651,10.302484,0.724764,0.770957,...,20480.025457,0.735599,0.688991,0.041153,0.046601,0.047496,0.047932,0.042594,0.044622,0.006908
642,993012,Andromeda Galaxy (M31),0.822373,74.018880,27287.948235,0.829902,15.874337,10.302484,0.724764,0.770957,...,20480.025457,0.735599,0.688991,0.041153,0.046601,0.047496,0.047932,0.042594,0.044622,0.006908
698,994009,Andromeda Galaxy (M31),0.838323,71.627275,18470.436157,0.578029,14.421358,10.302484,0.724764,0.770957,...,20480.025457,0.735599,0.688991,0.041153,0.046601,0.047496,0.047932,0.042594,0.044622,0.006908


In [20]:
dtest = dtest.sort_index()

In [31]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import catboost as ctb
from catboost import Pool, CatBoostRegressor

In [32]:
X = dtrain.drop(['y','galaxy'], axis=1)
y = np.log1p(dtrain.y)

#lab  = LabelEncoder()
#lab.fit(X["trend_class"])
#X.trend_class  = lab.transform(X.trend_class)
#categorical_columns = ['trend_class']
#categorical_features_pos = [13]

X.columns

Index(['galactic year', 'existence expectancy index',
       'existence expectancy at birth', 'Gross income per capita',
       'Income Index', 'Expected years of education (galactic years)',
       'Mean years of education (galactic years)',
       'Intergalactic Development Index (IDI)', 'Education Index',
       'Intergalactic Development Index (IDI), Rank',
       'Population using at least basic drinking-water services (%)',
       'Population using at least basic sanitation services (%)', 'IQR',
       'y_max', 'y_max_log', 'y_mean', 'y_mean_log', 'y_median',
       'y_median_log', 'y_min', 'y_min_log', 'y_quan_log_25', 'y_quan_log_75',
       'y_quan_log_95', 'y_quan_log_99', 'y_std', 'y_std_log', 'y_var',
       'y_var_log', 'ei_mean', 'gross_mean', 'dev_mean', 'in_idx',
       'nan_y_quan_log_25', 'nan_y_quan_log_75', 'nan_y_quan_log_95',
       'nan_y_quan_log_99', 'nan_y_mean_log', 'nan_y_median_log',
       'nan_y_std_log'],
      dtype='object')

In [139]:
# select features
colls = ['galactic year',
       'y_max',  'y_mean', 'y_mean_log', 'y_median','y_median_log', 
       'y_min', 'y_min_log', 'y_quan_log_25', 'y_quan_log_75',
       'y_quan_log_95', 'y_quan_log_99', 'y_std', 'y_var',
       ]

In [140]:
# Play with folds and features
seed = 12405
errcb2=[]
fold=KFold(n_splits=10, shuffle=True, random_state = seed)#100 # 1000

X_test = Pool(data=dtest[colls])
y_preds_ctb = []

for train_index, valid_index in fold.split(X[colls],y):
    
    X_train, X_valid = X[colls].iloc[train_index], X[colls].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = Pool(data = X_train, label = y_train)
    valid_data = Pool(data = X_valid, label = y_valid)
    
    m2  = CatBoostRegressor(eval_metric='RMSE',
                            use_best_model=True,
                            iterations=1000, 
                            learning_rate = 0.1, 
                            random_seed= seed,
                            
                           )
    m2.fit(train_data,eval_set=valid_data, early_stopping_rounds=50,verbose=False)
    preds=m2.predict(X_valid)
    print("Fold err: ",np.sqrt(mean_squared_error(y_valid,preds)))
    
    errcb2.append(np.sqrt(mean_squared_error(y_valid,preds)))
    preds = m2.predict(X_test)
    y_preds_ctb.append(preds)

    
print("Mean RMSE:",np.mean(errcb2))
print("Mean RMSE std :",np.std(errcb2))

print("Mean Target:", np.expm1(np.mean(y_preds_ctb,0)).mean())
print("First 10 :",np.expm1(np.mean(y_preds_ctb,0))[:5])
feature_importance = m2.get_feature_importance(train_data, fstr_type = ctb.EFstrType.FeatureImportance, prettified = True)
feature_importance.head(10)

Fold err:  0.0033032316340132263
Fold err:  0.0029169962501751977
Fold err:  0.004045086583110195
Fold err:  0.0036034536010934063
Fold err:  0.0035624591731465336
Fold err:  0.0017109381478377804
Fold err:  0.0021177202137519797
Fold err:  0.0030041748410434186
Fold err:  0.0034693986988043744
Fold err:  0.0037283954189028094
Mean RMSE: 0.0031461854561878923
Mean RMSE std : 0.0006963382971376941
Mean Target: 0.0819217332638751
First 10 : [0.0424882  0.03911087 0.03899615 0.03921202 0.02197052]


,Feature Id,Importances
0,galactic year,15.362631
1,y_min,14.024434
2,y_median,13.314905
3,y_max,9.415945
4,y_quan_log_95,7.375874
5,y_quan_log_75,6.572777
6,y_quan_log_99,6.508322
7,y_mean_log,5.570137
8,y_quan_log_25,5.501470
9,y_std,4.437078


In [141]:
# Fit on full data without cross validation. Gives bettetr score on LB.
train_data = Pool(data=X[colls], label=y)

X_test = Pool(data=dtest[colls])

m2  = CatBoostRegressor(eval_metric='RMSE',
                        iterations=1000, 
                        learning_rate = 0.1, 
                        random_seed= seed,

                       )
m2.fit(train_data, early_stopping_rounds=50,verbose=False)

org_pred = m2.predict(X_test)

In [142]:
# convert from target log
predictions_org = np.expm1(org_pred)

In [143]:
predictions_org[:10]

array([0.04204932, 0.03900619, 0.03877629, 0.03914754, 0.02203057,
       0.03277167, 0.02824643, 0.04184927, 0.04118303, 0.04142853])

In [144]:
predictions_org.mean()

0.08197267185152923

In [990]:
# Optimization task
from scipy import optimize


def objective(x, coeffs):

    return -np.sum(x*coeffs)

def apply_sum_constraint(inputs):
    """Sum of inputs should be 50000"""
    return 49999.9999 - np.sum(inputs) 

def apply_expectancy_constraint(inputs):
    """For rows with EI<0.7 constraint should be at least 5000"""
    total = np.sum(inputs[:66]) - 5000
    return total


coeffs = np.array((-np.log(predictions_org+0.01)+3)**2/1000)

x0=np.ones(890)*100

cons = (
        {'type':'ineq',
            'fun' : apply_sum_constraint
            },
        {'type':'ineq',
            'fun' : apply_expectancy_constraint
            })

bnd = tuple((0,100) for x in x0)

results = optimize.minimize(objective,
                             x0,
                             bounds=bnd,
                             args=coeffs,
                             constraints=cons,
                             method='SLSQP',
                             options={'disp':True,
                                      'maxiter':200
                                      })

Iteration limit exceeded    (Exit mode 9)
            Current function value: -1837.5080517856466
            Iterations: 201
            Function evaluations: 179292
            Gradient evaluations: 201


In [992]:
test_id  = test.index
d = {'Index': test_id, 'pred': predictions_org, 'opt_pred': results.x}
sub = pd.DataFrame(data=d)
sub = sub[['Index', 'pred','opt_pred']]
sub.to_csv('pred_nofolds6.csv', index=False)
sub.head()

,Index,pred,opt_pred
0,0,0.042248,100.0
1,1,0.038980,100.0
2,2,0.038524,100.0
3,3,0.039303,100.0
4,4,0.022019,100.0
